# Libraries

In [2]:
import json
import re
import operator

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from pymystem3 import Mystem
import pickle

%matplotlib inline

# Helper functions

In [3]:
m = Mystem()
stopWords = set(stopwords.words("russian"))
badwords = [u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
            u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
            u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
            u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
            u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
            u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок',u'в',
            u'б',u'г',u'д',u'е',u'ж',u'з',u'й',u'к',u'л',u'ф',u'н',u'о',u'п',u'р',u'с',u'т',u'ч',u'ц',u'ч',
            u'ш',u'щ',u'ь',u'ъ',u'ы',u'э','ю']

for word in badwords:
    stopWords.add(word)

In [4]:
def Lemmatisator(data):
    lemma = m.lemmatize(data)
    return ''.join(lemma)

def stopwordsEleminator(data, negation=True): #True to leave all не, ни negations
    for word in stopWords:
        if(word == u'не' or word == u'ни') and (negation == True):
            continue
        if word in data:
            for i in range(data.count(word)):
                data.remove(word)
    return data

def dataCleaner(data, negationConcat = True): # False if clear all, True if clear and concatinate negations
    cleanedData = re.sub("[^а-яА-ЯЁё]"," ", data) # leave only russian text
    cleanedData = Lemmatisator(cleanedData)
    cleanedData = cleanedData.lower().split()
    cleanedData = stopwordsEleminator(cleanedData,negationConcat)
    
    if negationConcat == True:
        for i in range (0, len(cleanedData)-1):
            if(cleanedData[i] == u'не' or cleanedData[i] == u'ни'):
                cleanedData[i+1]=('не'+cleanedData[i+1])
        cleanedData = stopwordsEleminator(cleanedData,False)

    return cleanedData

# Data loading

In [5]:
depression_posts = pd.read_csv('suicuide_data_V2.csv')
group_posts = pd.read_csv('group_posts.csv')
user_posts = pd.read_csv('user_posts.csv')

In [6]:
depression_posts.head(1)

,age,name,text,person_id,response
0,32.0,404,"Когда-то я был добрым романтиком, который стре...",1,"['404, поверьте, у каждого случается - не гото..."
1,28.0,Птица,Здраствуйте! Я каждый день просыпаюсь с мыслью...,2,"['Здравствуйте!Вот я сегодня прочла,что есть т..."


In [10]:
group_posts.head(1)

,created,domain,profile_type,profile_url,text,url
0,2019-03-22T21:35:20Z,vk.com,group,https://www.vk.com/public100919187,Токаев поручил правительству и акимам принять...,https://www.vk.com/wall-100919187_52089


In [11]:
user_posts.head(1)

,created,domain,profile_type,profile_url,text,url
0,2019-03-23T14:46:02Z,vk.com,user,https://www.vk.com/id289730685,"[id264842453|Нургуль], на фото взрослая, серье...",https://www.vk.com/wall-115351633_168996


In [12]:
depression_posts['label'] = 1
group_posts['label'] = 0
user_posts['label'] = 0

In [15]:
df = pd.concat([depression_posts[['text', 'label']], group_posts[['text', 'label']], user_posts[['text', 'label']]])

In [17]:
df.head(1)

,text,label
0,"Когда-то я был добрым романтиком, который стре...",1


# Cleaning

In [21]:
df['cleaned_text'] = df['text'].apply(lambda x: dataCleaner(str(x)))

In [22]:
df.head(1)

,text,label,cleaned_text
0,"Когда-то я был добрым романтиком, который стре...",1,"[добрый, романтик, стремиться, помогать, попро..."


In [24]:
df.to_csv('clean_text.csv', index_label=False)

In [7]:
df.cleaned_text.iloc[0]

"['добрый', 'романтик', 'стремиться', 'помогать', 'попросить', 'жалко', 'верить', 'человечность', 'однако', 'весь', 'фальший', 'суррогатный', 'дружба', 'непонятный', 'отношение', 'девушка', 'клясться', 'любить', 'итог', 'изменять', 'уходить', 'появляться', 'бесперспективный', 'попросту', 'достаточный', 'достаток', 'собственный', 'жилье', 'родитель', 'шизофрения', 'никто', 'поддерживать', 'поддерживать', 'итак', 'отдавать', 'кусочек', 'каждый', 'потерять', 'свой', 'душевный', 'тепло', 'становиться', 'безразлично', 'апатия', 'бывший', 'девушка', 'всякий', 'усилие', 'находить', 'новый', 'сразу', 'жениться', 'возить', 'разный', 'страна', 'мир', 'браться', 'любой', 'работа', 'показывать', 'плохой', 'часто', 'обманывать', 'недоплачивать', 'оплачивать', 'вовсе', 'простой', 'неудачник', 'потерять', 'смысл', 'жизнь', 'ситуация', 'спасать', 'внешность', 'ужасный', 'попросту', 'урод', 'часто', 'завидовать', 'везти', 'внешность', 'любить', 'даваться', 'легко', 'заниматься', 'свой', 'любимый', 'дел